In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns


Matplotlib is building the font cache; this may take a moment.


In [2]:
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel

In [3]:
from sklearn.metrics import average_precision_score

In [4]:
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [5]:
from sklearn.metrics import balanced_accuracy_score

In [6]:
Health_data = pd.read_csv('train.csv')
Health_data.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [7]:
Health_data.dtypes

id                        int64
Gender                   object
Age                       int64
Driving_License           int64
Region_Code             float64
Previously_Insured        int64
Vehicle_Age              object
Vehicle_Damage           object
Annual_Premium          float64
Policy_Sales_Channel    float64
Vintage                   int64
Response                  int64
dtype: object

In [8]:
Health_data = Health_data.drop(['id'],axis=1)

In [9]:
Health_data["Region_Code"].value_counts()

28.0    106415
8.0      33877
46.0     19749
41.0     18263
15.0     13308
30.0     12191
29.0     11065
50.0     10243
3.0       9251
11.0      9232
36.0      8797
33.0      7654
47.0      7436
35.0      6942
6.0       6280
45.0      5605
37.0      5501
18.0      5153
48.0      4681
14.0      4678
39.0      4644
10.0      4374
21.0      4266
2.0       4038
13.0      4036
7.0       3279
12.0      3198
9.0       3101
27.0      2823
32.0      2787
43.0      2639
17.0      2617
26.0      2587
25.0      2503
24.0      2415
38.0      2026
0.0       2021
16.0      2007
31.0      1960
23.0      1960
20.0      1935
49.0      1832
4.0       1801
34.0      1664
19.0      1535
22.0      1309
40.0      1295
5.0       1279
1.0       1008
44.0       808
42.0       591
52.0       267
51.0       183
Name: Region_Code, dtype: int64

In [12]:
Region_code_category = pd.cut(Health_data.Region_Code,bins=[0,10,20,30,40,50,60],labels=['RC_1-10','RC_11-20','RC_21-30','RC_31-40','RC_41-50','RC_51-60'])
Health_data.insert(4,'Region_code_category',Region_code_category)

In [104]:
Health_data.head()

,Gender,Age,Driving_License,Region_Code,Region_code_category,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,44,1,28.0,RC_21-30,0,> 2 Years,Yes,40454.0,26.0,217,1
1,Male,76,1,3.0,RC_1-10,0,1-2 Year,No,33536.0,26.0,183,0
2,Male,47,1,28.0,RC_21-30,0,> 2 Years,Yes,38294.0,26.0,27,1
3,Male,21,1,11.0,RC_11-20,1,< 1 Year,No,28619.0,152.0,203,0
4,Female,29,1,41.0,RC_41-50,1,< 1 Year,No,27496.0,152.0,39,0


In [105]:
Health_data["Region_code_category"].value_counts()

RC_21-30    147534
RC_41-50     71847
RC_1-10      68288
RC_11-20     47699
RC_31-40     43270
RC_51-60       450
Name: Region_code_category, dtype: int64

In [106]:
Health_data["Policy_Sales_Channel"].value_counts()

152.0    134784
26.0      79700
124.0     73995
160.0     21779
156.0     10661
          ...  
144.0         1
149.0         1
84.0          1
143.0         1
43.0          1
Name: Policy_Sales_Channel, Length: 155, dtype: int64

In [13]:
Policy_channel_category = pd.cut(Health_data.Policy_Sales_Channel,bins=[0,45,140,180],labels=['Policy_1-45','Policy_45-140','Policy_140-180'])
Health_data.insert(10,'Policy_channel_category',Policy_channel_category)

In [108]:
Health_data.head()

,Gender,Age,Driving_License,Region_Code,Region_code_category,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Policy_channel_category,Vintage,Response
0,Male,44,1,28.0,RC_21-30,0,> 2 Years,Yes,40454.0,26.0,Policy_1-45,217,1
1,Male,76,1,3.0,RC_1-10,0,1-2 Year,No,33536.0,26.0,Policy_1-45,183,0
2,Male,47,1,28.0,RC_21-30,0,> 2 Years,Yes,38294.0,26.0,Policy_1-45,27,1
3,Male,21,1,11.0,RC_11-20,1,< 1 Year,No,28619.0,152.0,Policy_140-180,203,0
4,Female,29,1,41.0,RC_41-50,1,< 1 Year,No,27496.0,152.0,Policy_140-180,39,0


In [109]:
Health_data["Policy_channel_category"].value_counts()

Policy_140-180    189829
Policy_1-45        98688
Policy_45-140      92592
Name: Policy_channel_category, dtype: int64

In [14]:
Health_data = Health_data.drop(['Region_Code'],axis=1)
Health_data = Health_data.drop(['Policy_Sales_Channel'],axis=1)

In [15]:
Health_data.head()

,Gender,Age,Driving_License,Region_code_category,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_channel_category,Vintage,Response
0,Male,44,1,RC_21-30,0,> 2 Years,Yes,40454.0,Policy_1-45,217,1
1,Male,76,1,RC_1-10,0,1-2 Year,No,33536.0,Policy_1-45,183,0
2,Male,47,1,RC_21-30,0,> 2 Years,Yes,38294.0,Policy_1-45,27,1
3,Male,21,1,RC_11-20,1,< 1 Year,No,28619.0,Policy_140-180,203,0
4,Female,29,1,RC_41-50,1,< 1 Year,No,27496.0,Policy_140-180,39,0


In [16]:
Health_data = pd.get_dummies(Health_data,columns=['Gender'],drop_first=True)
Health_data = pd.get_dummies(Health_data,columns=['Vehicle_Age'],drop_first=True)
Health_data = pd.get_dummies(Health_data,columns=['Vehicle_Damage'],drop_first=True)

In [113]:
Health_data.dtypes

Age                           int64
Driving_License               int64
Region_code_category       category
Previously_Insured            int64
Annual_Premium              float64
Policy_channel_category    category
Vintage                       int64
Response                      int64
Gender_Male                   uint8
Vehicle_Age_< 1 Year          uint8
Vehicle_Age_> 2 Years         uint8
Vehicle_Damage_Yes            uint8
dtype: object

In [17]:
Health_data = pd.get_dummies(Health_data,columns=['Region_code_category'],drop_first=True)
Health_data = pd.get_dummies(Health_data,columns=['Policy_channel_category'],drop_first=True)

In [18]:
Health_data.head()

,Age,Driving_License,Previously_Insured,Annual_Premium,Vintage,Response,Gender_Male,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_Yes,Region_code_category_RC_11-20,Region_code_category_RC_21-30,Region_code_category_RC_31-40,Region_code_category_RC_41-50,Region_code_category_RC_51-60,Policy_channel_category_Policy_45-140,Policy_channel_category_Policy_140-180
0,44,1,0,40454.0,217,1,1,0,1,1,0,1,0,0,0,0,0
1,76,1,0,33536.0,183,0,1,0,0,0,0,0,0,0,0,0,0
2,47,1,0,38294.0,27,1,1,0,1,1,0,1,0,0,0,0,0
3,21,1,1,28619.0,203,0,1,1,0,0,1,0,0,0,0,0,1
4,29,1,1,27496.0,39,0,0,1,0,0,0,0,0,1,0,0,1


In [19]:
train_health,test_health = train_test_split(Health_data,test_size=0.2,random_state=20)
train_health.head()

,Age,Driving_License,Previously_Insured,Annual_Premium,Vintage,Response,Gender_Male,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_Yes,Region_code_category_RC_11-20,Region_code_category_RC_21-30,Region_code_category_RC_31-40,Region_code_category_RC_41-50,Region_code_category_RC_51-60,Policy_channel_category_Policy_45-140,Policy_channel_category_Policy_140-180
136887,34,1,0,25285.0,205,0,0,0,0,1,0,0,0,0,0,0,1
11417,50,1,0,35252.0,54,0,1,0,0,1,0,1,0,0,0,0,0
290365,46,1,0,29641.0,114,1,0,0,0,1,0,0,0,0,0,1,0
274206,25,1,1,2630.0,93,0,0,1,0,0,0,0,0,1,0,0,1
282784,66,1,0,52920.0,67,1,0,0,0,1,0,1,0,0,0,0,0


In [117]:
train_health.shape

(304887, 17)

In [20]:
train_health_y =  train_health['Response']
train_health_x = train_health.drop(['Response'],axis =1)

In [119]:
rf1model = RandomForestClassifier(n_estimators=100,max_features=4)

In [120]:
rf1model.fit(train_health_x,train_health_y)

RandomForestClassifier(max_features=4)

In [121]:
test_health_y =  test_health['Response']
test_health_x = test_health.drop(['Response'],axis =1)
y_pred = rf1model.predict(test_health_x)

In [122]:
print(confusion_matrix(test_health_y,y_pred))
print(classification_report(test_health_y,y_pred))

[[64038  2836]
 [ 7937  1411]]
              precision    recall  f1-score   support

           0       0.89      0.96      0.92     66874
           1       0.33      0.15      0.21      9348

    accuracy                           0.86     76222
   macro avg       0.61      0.55      0.56     76222
weighted avg       0.82      0.86      0.83     76222



In [123]:
balanced_accuracy_score(test_health_y,y_pred)

0.5542666335296713

In [127]:
Sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=20, n_jobs=-1),threshold=0.01)

In [128]:
Sel.fit(train_health_x,train_health_y)

SelectFromModel(estimator=RandomForestClassifier(n_jobs=-1, random_state=20),
                threshold=0.01)

In [129]:
Sel.get_support()

array([ True, False,  True,  True,  True, False,  True, False,  True,
       False, False, False, False, False, False,  True])

In [130]:
Selected_Features = train_health_x.columns[(Sel.get_support())]

In [131]:
len(Selected_Features)

7

In [132]:
print(Selected_Features)

Index(['Age', 'Previously_Insured', 'Annual_Premium', 'Vintage',
       'Vehicle_Age_< 1 Year', 'Vehicle_Damage_Yes',
       'Policy_channel_category_Policy_140-180'],
      dtype='object')


In [133]:
new_train = train_health_x[Selected_Features]

In [134]:
new_train.head()

,Age,Previously_Insured,Annual_Premium,Vintage,Vehicle_Age_< 1 Year,Vehicle_Damage_Yes,Policy_channel_category_Policy_140-180
136887,34,0,25285.0,205,0,1,1
11417,50,0,35252.0,54,0,1,0
290365,46,0,29641.0,114,0,1,0
274206,25,1,2630.0,93,1,0,1
282784,66,0,52920.0,67,0,1,0


In [135]:
model2 =RandomForestClassifier(n_estimators=100,max_features=3)
model2.fit(new_train,train_health_y)

RandomForestClassifier(max_features=3)

In [136]:
new_test_x =test_health_x[Selected_Features]

In [137]:
y_pred2 = model2.predict(new_test_x)

In [138]:
print(confusion_matrix(test_health_y,y_pred2))
print(classification_report(test_health_y,y_pred2))

[[63451  3423]
 [ 7824  1524]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     66874
           1       0.31      0.16      0.21      9348

    accuracy                           0.85     76222
   macro avg       0.60      0.56      0.57     76222
weighted avg       0.82      0.85      0.83     76222



In [139]:
balanced_accuracy_score(test_health_y,y_pred2)

0.5559218564538995

In [140]:
Health_data['Response'].value_counts()

0    334399
1     46710
Name: Response, dtype: int64

# undersampling

In [27]:
no_of_yes = len(Health_data[Health_data['Response'] == 1])

In [28]:
print(no_of_yes)

46710


In [29]:
No_indices = Health_data[Health_data.Response == 0].index

In [30]:
random_No_indices = np.random.choice(No_indices,no_of_yes, replace=False)

In [31]:
Yes_indices = Health_data[Health_data.Response == 1].index

In [32]:
under_sample_indices = np.concatenate([Yes_indices,random_No_indices])

In [33]:
under_sample_health = Health_data.loc[under_sample_indices]

In [34]:
train_health_under,test_health_under = train_test_split(under_sample_health,test_size=0.2,random_state=20)
train_health_under.head()

,Age,Driving_License,Previously_Insured,Annual_Premium,Vintage,Response,Gender_Male,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_Yes,Region_code_category_RC_11-20,Region_code_category_RC_21-30,Region_code_category_RC_31-40,Region_code_category_RC_41-50,Region_code_category_RC_51-60,Policy_channel_category_Policy_45-140,Policy_channel_category_Policy_140-180
2390,26,1,1,25063.0,80,0,1,1,0,0,1,0,0,0,0,0,1
188559,21,1,1,2630.0,172,0,0,1,0,1,0,1,0,0,0,0,1
79841,45,1,0,43095.0,292,0,0,0,0,1,0,0,0,0,0,0,0
227751,50,1,0,29402.0,242,1,0,0,0,1,0,1,0,0,0,1,0
97318,23,1,0,47122.0,261,0,0,1,0,1,0,1,0,0,0,0,1


In [35]:
train_under_y =  train_health_under['Response']
train_under_x = train_health_under.drop(['Response'],axis =1)

In [36]:
# Create a random forest classifier
Under_rf = RandomForestClassifier(n_estimators=100, random_state=20, n_jobs=-1)

# Train the classifier
Under_rf.fit(train_under_x,train_under_y)

RandomForestClassifier(n_jobs=-1, random_state=20)

In [37]:
feat_labels = train_health_under.columns 
for feature in zip(feat_labels, Under_rf.feature_importances_):
    print(feature)

('Age', 0.15712658855145967)
('Driving_License', 0.0008494545219666529)
('Previously_Insured', 0.15344855102648997)
('Annual_Premium', 0.19512661327026518)
('Vintage', 0.2309906743095577)
('Response', 0.009984381771638516)
('Gender_Male', 0.02581263938448775)
('Vehicle_Age_< 1 Year', 0.004507082019313801)
('Vehicle_Age_> 2 Years', 0.17688021490460662)
('Vehicle_Damage_Yes', 0.004402751750916771)
('Region_code_category_RC_11-20', 0.005513188850731642)
('Region_code_category_RC_21-30', 0.004315633427357943)
('Region_code_category_RC_31-40', 0.004411863455288443)
('Region_code_category_RC_41-50', 0.0005233657619816949)
('Region_code_category_RC_51-60', 0.007285718717009931)
('Policy_channel_category_Policy_45-140', 0.018821278276927752)


In [39]:
sfm = SelectFromModel(Under_rf, threshold=0.01)

# Train the selector
sfm.fit(train_under_x, train_under_y)

SelectFromModel(estimator=RandomForestClassifier(n_jobs=-1, random_state=20),
                threshold=0.01)

In [40]:
for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])

Age
Previously_Insured
Annual_Premium
Vintage
Gender_Male
Vehicle_Age_> 2 Years
Policy_channel_category_Policy_45-140


In [41]:
test_under_y =  test_health_under['Response']
test_under_x = test_health_under.drop(['Response'],axis =1)

In [42]:
Selected_Features = train_under_x.columns[(sfm.get_support())] 
X_important_train = train_under_x[Selected_Features] 
X_important_test = test_under_x[Selected_Features]

In [43]:
rf_important = RandomForestClassifier(n_estimators=100, random_state=20, n_jobs=-1)

# Train the new classifier on the new dataset containing the most important features
rf_important.fit(X_important_train, train_under_y)

RandomForestClassifier(n_jobs=-1, random_state=20)

In [44]:
y_important_pred = rf_important.predict(X_important_test)

In [45]:
print(confusion_matrix(test_under_y,y_important_pred))
print(classification_report(test_under_y,y_important_pred))

[[6371 2908]
 [1662 7743]]
              precision    recall  f1-score   support

           0       0.79      0.69      0.74      9279
           1       0.73      0.82      0.77      9405

    accuracy                           0.76     18684
   macro avg       0.76      0.75      0.75     18684
weighted avg       0.76      0.76      0.75     18684



In [46]:
balanced_accuracy_score(test_under_y,y_important_pred)

0.7549448231872041

In [47]:
test_health_x.head()

,Age,Previously_Insured,Annual_Premium,Vintage,Gender_Male,Vehicle_Age_> 2 Years,Policy_channel_category_Policy_45-140
296947,25,1,38036.0,154,0,0,0
312201,45,1,33735.0,120,1,0,1
342199,35,1,41332.0,82,1,0,0
190781,27,0,39911.0,54,1,0,0
129148,43,0,2630.0,137,0,0,1


In [162]:
X_general_test = test_health_x[Selected_Features]

In [163]:
X_general_test

,Age,Previously_Insured,Annual_Premium,Vintage,Gender_Male,Vehicle_Age_< 1 Year,Vehicle_Damage_Yes,Policy_channel_category_Policy_140-180
296947,25,1,38036.0,154,0,1,0,1
312201,45,1,33735.0,120,1,0,0,0
342199,35,1,41332.0,82,1,0,0,1
190781,27,0,39911.0,54,1,1,0,1
129148,43,0,2630.0,137,0,0,0,0
...,...,...,...,...,...,...,...,...
342046,50,0,55007.0,84,0,0,1,0
239565,62,0,42063.0,274,1,0,0,0
301859,46,0,2630.0,168,1,0,1,1
160431,48,0,47618.0,42,0,0,1,0


In [164]:
y_general_predict = rf_important.predict(X_general_test)

In [165]:
print(confusion_matrix(test_health_y,y_general_predict))
print(classification_report(test_health_y,y_general_predict))

[[48539 18335]
 [  335  9013]]
              precision    recall  f1-score   support

           0       0.99      0.73      0.84     66874
           1       0.33      0.96      0.49      9348

    accuracy                           0.76     76222
   macro avg       0.66      0.84      0.66     76222
weighted avg       0.91      0.76      0.80     76222



In [166]:
balanced_accuracy_score(test_health_y,y_general_predict)

0.8449955666759561

In [48]:
import joblib

In [49]:
# save the model to disk
filename = 'RandomClassifier_model.joblib'
joblib.dump(rf_important, filename)

['RandomClassifier_model.joblib']

In [50]:
test_health_y =  test_health['Response']
test_health_x = test_health.drop(['Response'],axis =1)

In [51]:
test_health_x =test_health_x[['Age','Previously_Insured','Annual_Premium','Vintage','Gender_Male','Vehicle_Age_> 2 Years'
,'Policy_channel_category_Policy_45-140']]

In [52]:
filename = 'RandomClassifier_model.joblib'
loaded_model = joblib.load(filename)
result = loaded_model.score(test_health_x,test_health_y)
print(result)

0.8512634147621422
